In [1]:
def build_and_compile_model(norm):
    model = keras.Sequential([
        norm,
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001))
    return model

In [2]:
import three_desc_model as exponential_new
import numpy as np
import pandas as pd
from cutoffs import Polynomial
from scipy.optimize import minimize,fmin
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
dataset_full = pd.read_csv('all_dataset.csv', index_col=0)
symbol="Siemens"
#dataset_pos=np.empty(shape=(0, 2))
#dataset_fp=np.empty(shape=(0, 3))
#dataset_ws=np.empty(shape=(0, 1))
k=0
count=0
dataset_full["IDnum"]=pd.Series()
for i in range(len(dataset_full)):
    dataset_full.at[i,"IDnum"]=k
    count = count +1
    if (count==dataset_full["Num_tot_turb"].iloc[i]):
        k=k+1
        count=0
dataset_full["Num_tot_turb"]=dataset_full["Num_tot_turb"].astype(int)
dataset_full["Turb_num"]=dataset_full["Turb_num"].astype(int)
dataset_full["IDnum"]=dataset_full["IDnum"].astype(int)
numsims=dataset_full["IDnum"].iloc[-1]+1
turb = [symbol]
nl=exponential_new.NeighborlistCalculator(cutoff=4001,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)



/var/folders/gr/z6l2h7kd4136nf0xqdtls40w0000gn/T/ipykernel_18327/980379466.py:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dataset_full["IDnum"]=pd.Series()


In [20]:
def f(x):
    Gs = {"Siemens": [{"type":"G2", "turbine":"Siemens","eta":x[0], "offset": 1.},
                  {"type":"G4", "elements":"Siemens","eta":x[1], "gamma":4., "zeta": 1.0},
                  {"type":"G6", "elements":"Siemens","eta":x[2], "gamma": 0.5, "zeta": 1.0}
                 ]}
    finpr=exponential_new.FingerprintCalculator(cutoff=4001,Gs=Gs,Rct=3000,delta_R=100,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)
    count=0
    dataset_fp=np.empty(shape=(0, 3))
    dataset_ws=np.empty(shape=(0, 1))
    for i in range(numsims):
        numturb=dataset_full["Num_tot_turb"].iloc[count]
        position = np.empty((numturb,2))
        ws = np.empty((numturb,1))
        fp = np.empty((numturb,3))
        for k in range(numturb):
            position[k,0]=dataset_full.at[count,"X_coord"]
            position[k,1]=dataset_full.at[count,"Y_coord"]
            ws[k,0]=dataset_full.at[count,"Ref_wind"]
            count = count+ 1
        neigh=nl.calculate(turb*numturb,position)
        fingerprints=finpr.calculate(turb*numturb,position,neigh,symbol)
        fingerprints=np.array(fingerprints)
        #dataset_pos=np.append(dataset_pos,position,axis=0)
        dataset_fp=np.append(dataset_fp,fingerprints,axis=0)
        dataset_ws=np.append(dataset_ws,ws,axis=0)
    dataset=np.concatenate((dataset_fp, dataset_ws),axis=1)
    dataset = pd.DataFrame(dataset, columns = ['Fingerprint(G2)','Fingerprint(G4)','Fingerprint(G6)','Ref_Wind_Speed'])
    train_dataset = dataset#.sample(frac=0.8, random_state=1)
    test_dataset = dataset.drop(train_dataset.index)
    train_features = train_dataset.copy()
    #test_features = test_dataset.copy()
    all_features = dataset.copy()
    train_labels = train_features.pop('Ref_Wind_Speed')
    #test_labels = test_features.pop('Ref_Wind_Speed')
    all_labels = all_features.pop('Ref_Wind_Speed')
    normalizer = preprocessing.Normalization(axis=-1)
    normalizer.adapt(np.array(train_features))
    dnn_model = build_and_compile_model(normalizer)
    history = dnn_model.fit(
        train_features, train_labels,
        validation_split=0.2,
        verbose=0, epochs=20)
    all_predictions = dnn_model.predict(all_features).flatten()
    error_all=np.abs(all_predictions-all_labels)
    print(np.mean(error_all))
    return np.mean(error_all)

In [22]:
i=np.linspace(4,6,41)

for k in i:
    x0 = [k, 5., 2.]
    f(x0)


0.09640851283760446
0.10781580213089262
0.19094084596739694
0.13789863622808274
0.13493021919255482


KeyboardInterrupt: 

In [6]:
x0 = [5.3, 5., 2.]
minimum = fmin(f, x0,disp=True)

In [11]:
x0 = [5.3, 5., 2.]
bounds= ((0,10),(0,10),(0,10))
res = minimize(f, x0, bounds= bounds, options={'disp':True})

1.7378330247534182
2.138951303256836
2.227571489108887
1.4762315766943361
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.73783D+00    |proj g|=  8.00000D+00
2.179893972810059
4.083277704013672
3.462219239963379
2.755338193668213
2.2166786210668947
2.2366204278601076
2.2720608728063967
1.7659029977453615
1.4710726754797365
2.8015570657385256
2.127349855197754
2.5706262605322268
2.1728320138586428
1.2192502038610842
1.5538668649328615
1.8310651795996096
1.8122301118505861
1.4014024751318361
2.0281128900183107
2.166484834445801
1.1366200428295903
3.0420002954138186
2.706223966373291
1.395622251539307
2.270212175144043
1.4100399010522464
1.550872804416504
1.6764559762609865
1.3722467439306643
1.9405574815405275
1.3890047090185549
2.0354313867224123
1.2271957414282229
2.7014284150732424
1.6872291581762697
1.4876914041174318
1.3162899034155275
1.55

In [ ]:
count=0
for i in range(numsims):
#    if dataset.loc[dataset['Number of Turbine in the Simulation'] == 2].reset_index(drop=True)
    numturb=dataset["Num_tot_turb"].iloc[count]
    print(count)
    position = np.empty((numturb,2))
    ws = np.empty((numturb,1))
    fp = np.empty((numturb,3))
    for k in range(numturb):
        position[k,0]=dataset.at[count,"X_coord"]
        position[k,1]=dataset.at[count,"Y_coord"]
        ws[k,0]=dataset.at[count,"Ref_wind"]
        count = count+ 1
    #print(position)
    neigh=nl.calculate(turb*numturb,position)
    fp=finpr.calculate(turb*numturb,position,neigh,symbol)
    fp=np.array(fp)
    print(fp)
    dataset_pos=np.append(dataset_pos,position,axis=0)
    dataset_fp=np.append(dataset_fp,fp,axis=0)
    dataset_ws=np.append(dataset_ws,ws,axis=0)
dataset=np.concatenate((dataset_fp, dataset_ws),axis=1)